In [1]:
import h5py
import numpy as np
from sklearn.model_selection import train_test_split

feature_file = '../preprocess/MSRII-c3d-features.h5'

with h5py.File(feature_file, 'r') as h5:
    X = np.array(h5['features'])
    Y = np.array(h5['labels']).reshape(X.shape[0])
    Y[Y > 0] = 1

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [16]:
%env KERAS_BACKEND=theano
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation


def FC1Net():
    model = Sequential()
    model.add(Dense(1, input_dim=4096, activation='sigmoid'))
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


def FC4Net():
    model = Sequential()
    model.add(Dense(256, input_dim=4096, init='uniform', activation='sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(256, init='uniform', activation='sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(256, init='uniform', activation='sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    return model

model_creator = FC4Net
model = model_creator()
model.fit(
    X_train, y_train,
    nb_epoch=100, batch_size=32,
    validation_data=(X_test, y_test), verbose=0);

env: KERAS_BACKEND=theano


In [3]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

estimator = KerasClassifier(build_fn=model_creator, nb_epoch=10, batch_size=32, verbose=0)
kfold = StratifiedKFold(n_splits=4, shuffle=True)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)

print("Results: %.2f%% (%.2f%%)" % (results.mean() * 100, results.std() * 100))

Results: 88.82% (3.19%)


In [4]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=model_creator, nb_epoch=10, batch_size=32, verbose=0)))
pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits=4, shuffle=True)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)

print("Results: %.2f%% (%.2f%%)" % (results.mean() * 100, results.std() * 100))

Results: 89.61% (1.24%)


In [17]:
loss, accuracy = model.evaluate(X_test, y_test, batch_size=32, verbose=0)
print('Results: %.2f%%' % (accuracy * 100))

Results: 93.01%


In [6]:
model.save_weights('FC4Net_weights.h5')